In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
from PIL import Image
import os, glob
import numpy as np

In [ ]:
# Helper function to preprocess the image so that it can be inputted in MobileNetV2
def get_imagenet_label(probs):
  # Helper function to extract str label from probability vector
  return decode_predictions(probs, top=1)[0][0] # probs(softmax output) -> [[('n02099712', 'Labrador_retriever', 0.41818538)]]

def convert_label_id_to_idx(true_y_id, class_num):
  for i in range(class_num):
    label = tf.one_hot(i, class_num)
    label = tf.reshape(label, (1, class_num))
    label = np.array(label)

    if true_y_id == get_imagenet_label(label)[0]:
      print(get_imagenet_label(label)[1])
      break
      
  return label

def preprocess(image, size):
  """
  :params image: image with shape (H,W,C)
  :params size: image size that you want to resize
  """
  image = np.array(image) # convert to np array
  image = tf.cast(image, tf.float32) # casting pixel values into float numbers
  image = tf.image.resize(image, (size[0], size[1])) # resize image size according to the network
  image = tf.keras.applications.inception_v3.preprocess_input(image) # normalize pixel values into the range [-1, 1]
  image = image[None, ...] # adds batch dimension to the input (H, W, C) -> (B, H, W, C)

  return image

def calc_grad_fgsm(image, label, model):
  """
  :params image: image with shape (H,W,C)
  :params label: one-hot encoded ground truth to calculate the loss
  :params model: pre-trained neural network on ImageNet
  """
  loss_object = tf.keras.losses.CategoricalCrossentropy() # loss function for image multi-classification task

  with tf.GradientTape() as tape:
    tape.watch(image)
    prediction = model(image)
    loss = loss_object(label, prediction) # comparing prob distribution of ground truth and prediction
    gradient = tape.gradient(loss, image) # get the gradients of the loss w.r.t to the input image.
  
  signed_grad = tf.sign(gradient) # get the sign of the gradients to create the perturbation

  return signed_grad

def display_images(image, model, description=None): 
  """
  :params image: image with shape (H,W,C)
  :params model: pre-trained neural network on ImageNet
  :params description: one-hot encoded ground truth to calculate the loss
  """
  _, label, confidence = get_imagenet_label(model.predict(image)) # get the label name and the confidence from softmax output predictions
  plt.figure()
  plt.imshow(image[0]*0.5 + 0.5)

  if description is None:
    plt.title('Original \n {} : {:.2f}% Confidence'.format(label, confidence*100))
  else:
    plt.title('{} \n {} : {:.2f}% Confidence'.format(description, label, confidence*100))

  plt.show()

In [ ]:
epsilons = [0.01, 0.1, 0.15, 0.2] # different values of epsilon
descriptions = [('Epsilon = {:0.3f}'.format(eps) if eps else 'Input') for eps in epsilons]
class_num = 1000
data_dir = '/content/drive/My Drive/datasets/tiny-imagenet-200/train/n04285008/images'
true_y_id = data_dir.split('/')[-2]

dirFiles = glob.glob(os.path.join(data_dir, '*.JPEG'))
dirFiles.sort()

# load the model and freeze the parameters
model = tf.keras.applications.InceptionV3(include_top=True, weights='imagenet', classifier_activation='softmax')
model.trainable = False

# ImageNet labels function
decode_predictions = tf.keras.applications.inception_v3.decode_predictions

# get ground truth one-hot vector for each image
true_y_vector = convert_label_id_to_idx(true_y_id, class_num)

sports_car


In [ ]:
# vanilla FSGM
for im_path in dirFiles[:2]:

  # preprocess the image so that it could be fed into the model
  image = Image.open(im_path)  
  x = preprocess(image, (299, 299))

  # get the perturbation (signed gradients of loss w.r.t to input image) which will be added to the image 
  perturbations = calc_grad_fgsm(x, true_y_vector, model) 
  
  # display classification result of original image
  display_images(x, model) 

  # for each epsilon value, add perturbation to original image
  for i, eps in enumerate(epsilons):
    adv_x = x + eps * perturbations
    adv_x = tf.clip_by_value(adv_x, -1, 1)
    display_images(adv_x, model, descriptions[i])

In [ ]:
epochs = 4

for im_path in dirFiles[:5]:

  # preprocess the image so that it could be fed into the model
  image = Image.open(im_path)  
  x = preprocess(image, (299, 299))

  # display classification result of original image
  display_images(x, model) 
  
  # basic iterative method: run n epochs to find the adversarial image incurring largest loss possible with step size epsilon
  for i, eps in enumerate(epsilons):
    adv_x = x

    for j in range(1, epochs):
      print('epoch: {}'.format(j))
      perturbations = calc_grad_fgsm(adv_x, true_y_vector, model)  # get the perturbation(signed gradients of loss w.r.t to input image) which will be added to the image
      adv_x = adv_x + eps * perturbations

    adv_x = tf.clip_by_value(adv_x, -1, 1)
    display_images(adv_x, model, descriptions[i])

Output hidden; open in https://colab.research.google.com to view.